<a href="https://colab.research.google.com/github/priyariyyer/AIML_Projects/blob/main/Question_Answer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Change the runtime to T-4 GPU

## Data Gathering

In [3]:
# import requisite libraries
import langchain

In [4]:
# install Langchain Community
!pip install langchain_community

In [6]:
# install pypdf for reading pdf doc
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 22.7 MB/s eta 0:00:00


In [8]:
from langchain_community.document_loaders import PyPDFLoader

In [10]:
loader = PyPDFLoader('/content/budget_speech.pdf') #loads all pages of the document
all_pages = loader.load_and_split() #split the pages
all_pages

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': '/content/budget_speech.pdf', 'total_pages': 60, 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2025'),
 Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': '/content/budget_speech.pdf', 'total_pages': 60, 'page': 2, 'page_label': '3'}, page_content='CONTENTS \n \nPART – A \n Page No. \nIntroduction 1 \nBudget Theme 1 \nAgriculture as the 1st engine 3 \nMSMEs as the 2nd engine 6 \nInvestment as the 3rd engine 8 \nA. Investing in People 8 \nB. Investing in the Economy 10 \nC. Investing in Innovation 14 \nExports a

## Chunking

In [12]:
#Each model has its own tokenization mechanism
#Word to token ratio helps define chunk size. For example, 1 word = 1 token or set of 3 words = 1 token, etc.
#For this we need tokenizer which can be downloaded
from transformers import AutoTokenizer

In [13]:
#all LLMs today are Causal Language Model
model_name = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = 'gptq_model-4bit-128g'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [14]:
#use the tokenizer to get token count by applying on each page
token_count = [len(tokenizer(page.page_content, return_tensors='pt').input_ids.cuda()[0]) for page in all_pages]

In [15]:
token_count

[60,
 195,
 362,
 465,
 525,
 599,
 653,
 578,
 595,
 541,
 570,
 591,
 583,
 522,
 494,
 480,
 494,
 530,
 487,
 546,
 230,
 503,
 648,
 539,
 596,
 553,
 617,
 550,
 651,
 342,
 235,
 205,
 619,
 594,
 634,
 610,
 576,
 472,
 463,
 449,
 596,
 632,
 587,
 578,
 554,
 542,
 226,
 423,
 822,
 644,
 522,
 484,
 488,
 551,
 570,
 485,
 368,
 380]

In [16]:
print('Min Token Count: ', min(token_count))
print('Max Token Count: ', max(token_count))
print('Average Token Count: ', sum(token_count)/len(token_count))

Min Token Count:  60
Max Token Count:  822
Average Token Count:  507.0344827586207


In [17]:
#to get the length of a token after breaking the given text when it reaches chunk size
def tokenizer_len(text):
  return len(tokenizer(text, return_tensors='pt').input_ids.cuda()[0])

# info:
#move your input IDs (and other tensors) to the GPU for faster processing.
#This is done by transferring the tensor to the CUDA device.
#CUDA (Compute Unified Device Architecture) is a parallel computing platform and API developed by NVIDIA.
#It allows developers to leverage the power of NVIDIA GPUs for computational tasks, including deep learning.


In [19]:
#create text splitter for creating chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20, length_function = tokenizer_len, separators=['\n\n', '\n',' '])
###
# The RecursiveCharacterTextSplitter works by attempting to split the text using a predefined set of characters.
# It starts with the first character (\n\n) and checks if the resulting chunks are within the specified size.
# If not, it moves to the next character (\n) and repeats the process.
# This continues until the chunks are within the desired size
# For example, given a chunk size of 100 characters, the splitter will first try to split the text by paragraphs (\n\n).
# If the resulting chunks are still too large, it will then try to split by new lines (\n),
# followed by spaces ( ), and finally by individual characters ("").
# This recursive approach ensures that the text is split into manageable chunks without losing the context
###

In [20]:
import hashlib #library to hash each chunk so that each chunk gets unique id
hasher = hashlib.md5()

In [21]:
from tqdm.auto import tqdm #tqdm is file organization library. used to read pages in a file organizer so that page hierarchy is maintained
chunks = []

#create chunks
for page in tqdm(all_pages):
  url = page.metadata['source']
  hasher.update(url.encode('utf-8'))
  uid = hasher.hexdigest()[:12] #take only first 12 characters of ID generated by hasher. This is done as Pinecone requires max 12 char len of id
  chunks_list = text_splitter.split_text(page.page_content)
  for i, chunk in enumerate(chunks_list):
    chunks.append({'id': f'{uid}-{i}', 'text': chunk, 'source': url})

len(chunks)


  0%|          | 0/58 [00:00<?, ?it/s]

114

In [22]:
chunks

[{'id': 'cae6ca4071f5-0',
  'text': 'GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2025',
  'source': '/content/budget_speech.pdf'},
 {'id': '90f6f359d522-0',
  'text': 'CONTENTS \n \nPART – A \n Page No. \nIntroduction 1 \nBudget Theme 1 \nAgriculture as the 1st engine 3 \nMSMEs as the 2nd engine 6 \nInvestment as the 3rd engine 8 \nA. Investing in People 8 \nB. Investing in the Economy 10 \nC. Investing in Innovation 14 \nExports as the 4th engine 15 \nReforms as the Fuel 16 \nFiscal Policy 18 \n \n \nPART – B \nIndirect taxes 20 \nDirect Taxes  23 \n \nAnnexure to Part-A 29 \nAnnexure to Part-B 31',
  'source': '/content/budget_speech.pdf'},
 {'id': '969632a1ddf0-0',
  'text': 'Budget 2025-2026 \n \nSpeech of \nNirmala Sitharaman \nMinister of Finance \nFebruary 1, 2025 \nHon’ble Speaker,  \n I present the Budget for 2025-26. \nIntroduction \n1. This Budget continues our Government’s efforts to: \na) accelerate growth,  \nb)

In [24]:
#view chunks in a dataframe format
import pandas as pd
chunk_df = pd.DataFrame(chunks)
chunk_df

,id,text,source
0,cae6ca4071f5-0,GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\...,/content/budget_speech.pdf
1,90f6f359d522-0,CONTENTS \n \nPART – A \n Page No. \nIntroduct...,/content/budget_speech.pdf
2,969632a1ddf0-0,Budget 2025-2026 \n \nSpeech of \nNirmala Sith...,/content/budget_speech.pdf
3,969632a1ddf0-1,Our development track record of the past 10 ye...,/content/budget_speech.pdf
4,f60f92997804-0,2 \n \ngrown in this period. We see the next ...,/content/budget_speech.pdf
...,...,...,...
109,b72d180e1e9e-1,applicable to a case where search has been ini...,/content/budget_speech.pdf
110,92bb10814016-0,55 \n \nCentral Government may issue directio...,/content/budget_speech.pdf
111,92bb10814016-1,• It is proposed that any order imposing a pen...,/content/budget_speech.pdf
112,ee6ce8dcf8ee-0,56 \n \n11. Clarification regarding commencem...,/content/budget_speech.pdf


## Embedding

In [25]:
#Closed source models have their own embedding model
#Since we are using open source models, we can choose any embedding model
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
#check using a sample text
sample = 'Hello all! this is a sample text for testing sentence embedding of this sample.'
embed_len = len(embedding_model.encode(sample))

In [28]:
#Prepare database to store embeddings
#Here we are using Pinecone vector database
!pip install pinecone-client==3.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.9/199.9 kB 14.8 MB/s eta 0:00:00


In [29]:
!pip install langchain-pinecone==0.0.3 #use pinecone using this library of Langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.45
    Uninstalling langsmith-0.3.45:
      Succes

In [31]:
embed_len

384

In [32]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='pcsk_3Ypxua_SEHLa7joNyFKfzwSzPPkxJ3BLirYfavRTRPhaYbCt52x1qjFHb6rAxFTWLrNLPf')
index_name = 'budgetspeech' #has to be in lowercase

#create table(index) in pinecone
pc.create_index(name=index_name,
  dimension=embed_len, #provide embedding len from embedding model
  metric='cosine', #provide similarity search mechanism
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

In [33]:
#get index info and its stats
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

##Indexing

In [35]:
type(chunk_df['text'])

pandas.core.series.Series

In [36]:

batch_size = 100 #chunks are embedded in batches to make it faster
metadata = []

for i in tqdm(range(0, len(chunk_df), batch_size)):
  i_end = min(len(chunk_df), i+batch_size)
  batch = chunk_df.iloc[i:i_end]
  metadata = [{
     'source': record['source'],
     'text' : record['text']
             } for j, record in batch.iterrows()]

  #encode the metadata
  embeds = embedding_model.encode(list(batch['text'])).tolist()
  print(len(embeds))

  ids = batch['id']
  index.upsert(vectors=zip(ids, embeds, metadata))



  0%|          | 0/2 [00:00<?, ?it/s]

100
14


In [37]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 114}},
 'total_vector_count': 114}